In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import numpy as np
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords

In [ ]:
import pandas as pd
training_set = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_set =  pd.read_csv('../input/commonlitreadabilityprize/test.csv')
training_set.head()

In [ ]:
english_punctuations = '''"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~'''
def remove_punctuations(sentence):
    translator = str.maketrans('', '', english_punctuations)
    return sentence.translate(translator)

In [ ]:
training_set['clean_excerpt'] = training_set['excerpt'].apply(lambda x:remove_punctuations(x))
test_set['clean_excerpt'] = test_set['excerpt'].apply(lambda x:remove_punctuations(x))

In [ ]:
stop_words  = stopwords.words('english')

In [ ]:
def remove_stop_words(sentence):
    zen = TextBlob(sentence)
    words = zen.words
    return " ".join([w for w in words if not w in stop_words and len(w) >= 2])

In [ ]:
#training_set['clean_excerpt'] = training_set['clean_excerpt'].apply(lambda x:remove_stop_words(x))
#test_set['clean_excerpt'] = test_set['clean_excerpt'].apply(lambda x:remove_stop_words(x))

In [ ]:
n_splits = 5
training_set['bin'] = pd.cut(training_set.target,20, labels=[i for i in range(20)])
skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
gen_skf = skf.split(training_set.id, y=training_set.bin)

In [ ]:
X_train = training_set['clean_excerpt']
Y_train = training_set['target']

In [ ]:
X_test = test_set['clean_excerpt']

# SVR 

In [ ]:
from sklearn.svm import SVR
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_train = X_train.toarray()
X_test =  vectorizer.transform(X_test)
X_test = X_test.toarray()

In [ ]:
from sklearn.metrics import mean_squared_error
cv_pred = np.zeros((X_train.shape[0],))
test_pred = np.zeros((X_test.shape[0],))

for fold, (idx_train, idx_val) in enumerate(gen_skf):
    SVR_model = SVR(C=10,gamma=0.1)
    SVR_model.fit(X_train[idx_train],Y_train.values[idx_train])
    Y_pred_SVR = SVR_model.predict(X_train[idx_val])
    cv_pred[idx_val]=Y_pred_SVR
    test_pred  +=  SVR_model.predict(X_test)/n_splits
    print(f'remse fold{fold}={mean_squared_error(Y_train.values[idx_val], Y_pred_SVR)}')

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_train, cv_pred)

# RF

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
#pipe = make_pipeline(TfidfVectorizer(),RandomForestRegressor())
#param_grid = {'randomforestregressor__n_estimators':[1000],
#             'randomforestregressor__max_features':['sqrt']}

In [ ]:
#RF_model = GridSearchCV(pipe, param_grid, cv=2)
#RF_model.fit(X_train.values,Y_train.values)

In [ ]:
#RF_model.best_estimator_

In [ ]:
#from sklearn.metrics import mean_squared_error
#Y_pred_RF = RF_model.predict(X_test.values)
#mean_squared_error(Y_test, Y_pred_RF)

In [ ]:
#Y_pred=Y_pred_RF*0.2 + Y_pred_SVR*0.8
#mean_squared_error(Y_test, Y_pred)

In [ ]:
#test_pred = svr_test_pred*0.8 + rf_test_pred*0.2

In [ ]:
del test_set['clean_excerpt']
del test_set['license']
del test_set['url_legal']
del test_set['excerpt']
test_set['target'] = test_pred

In [ ]:
test_set

In [ ]:
test_set.to_csv("submission.csv", index=False)